#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Input required from user

In [2]:
# BACI database filepath
BACI_file = "../Trade/BACI_HS17_Y2018_V202102.csv"

# Plastic HS codes filepath
PlasticsHS_file = "../Trade/HSPlastics_code.xlsx"

# Country code (check "CountryCode" sheet in Tool T1.4a)
cntr_code = 704

# Output: Country trade data
out_file = "../Trade/Vietnam_imp_exp.xlsx"

#### Run

In [3]:
#Load BACI trade data
df = pd.read_csv(BACI_file)

#Load file containing HS codes and plastic/packaging shares
df_plastic = pd.read_excel(PlasticsHS_file)

#Extract the 4 digit codes from the database
df_plastic_k4 = df_plastic.loc[df_plastic["k4"]<10000]

#Extract the 6 digits code from the database
df_plastic_k6 = df_plastic.loc[df_plastic["k4"]>10000]
df_plastic_k6 = df_plastic_k6.rename(columns={"k4":"k"})

#Drop year and monetary value
df = df.drop(columns=["t","v"])

#Extract Country data for export
df_country_exp = df.loc[df["i"]==cntr_code].copy()
df_country_exp = df_country_exp.drop(columns=["i","j"])

#Add 4 digits column
df_country_exp["k4"]=np.trunc(df_country_exp["k"]/100).astype(int)

#Merge country database with the list of plastic HS codes and shares of plastic intensity
df_country_exp_6 = pd.merge( df_country_exp, df_plastic_k6,on="k", how="inner")
df_country_exp_6["Trade Flow Code"] = 2
df_country_exp_6["Estimated plastic weight (kg)"] = df_country_exp_6["Plastic percentage"]*df_country_exp_6["q"]*1000

#Merge country database with the list of plastic HS codes and shares of plastic intensity
df_country_exp_4 = pd.merge( df_country_exp, df_plastic_k4, on = "k4", how="inner")
df_country_exp_4["Trade Flow Code"] = 2
df_country_exp_4["Estimated plastic weight (kg)"] = df_country_exp_4["Plastic percentage"]*df_country_exp_4["q"]*1000

#Extract Country data for import
df_country_imp = df.loc[df["j"]==cntr_code].copy()
df_country_imp = df_country_imp.drop(columns=["i","j"])

#Add 4 digits column
df_country_imp["k4"]=np.trunc(df_country_imp["k"]/100).astype(int)

#Merge country database with the list of plastic HS codes and shares of plastic intensity
df_country_imp_6 = pd.merge( df_country_imp, df_plastic_k6,on="k", how="inner")
df_country_imp_6["Trade Flow Code"] = 1
df_country_imp_6["Estimated plastic weight (kg)"] = df_country_imp_6["Plastic percentage"]*df_country_imp_6["q"]*1000

#Merge country database with the list of plastic HS codes and shares of plastic intensity
df_country_imp_4 = pd.merge( df_country_imp, df_plastic_k4,on="k4", how="inner")
df_country_imp_4["Trade Flow Code"] = 1
df_country_imp_4["Estimated plastic weight (kg)"] = df_country_imp_4["Plastic percentage"]*df_country_imp_4["q"]*1000

exp = df_country_exp_6["Estimated plastic weight (kg)"].sum()/1e6  + df_country_exp_4["Estimated plastic weight (kg)"].sum()/1e6

imp = df_country_imp_6["Estimated plastic weight (kg)"].sum() /1e6 + df_country_imp_4["Estimated plastic weight (kg)"].sum()/1e6

df_exp = df_country_exp_4.append(df_country_exp_6)
df_imp = df_country_imp_4.append(df_country_imp_6)

df_country = df_exp.append(df_imp)

df_country.to_excel(out_file, index = False)